# Splitting Wav Files By Channel

## Imports

In [134]:
from scipy.io import wavfile
import re
import os
import pandas as pd
import textgrids
import glob
from functions import *

# Wave Files 

I'm assuming that the file is made up of two channels (microphones) and has annotations as praat Textgrids. This step will split the wav files into two different audio files with proper names. 

In [109]:
# get the wav files
my_wav_files = get_wav_files()

In [112]:
# why can't i put this in a function? it refuses to find cwd for some reason

cwd = os.getcwd()
for item in my_wav_files:
    fs, data = wavfile.read(item)
    channel1, channel2 = get_file_names(item)
    wavfile.write(channel1, fs, data[:, 0])   # saving first column which corresponds to channel 1
    wavfile.write(channel2, fs, data[:, 1])   # saving second column which corresponds to channel 2
    os.replace(cwd + '/' + channel1, cwd + '/' + "split_wav_files_folder" + '/' + channel1)
    os.replace(cwd + '/' + channel2, cwd + '/' + "split_wav_files_folder" + '/' + channel2)

# Textgrids

In [116]:
## not sure why this isn't working... the functions file can't find this for some reason

# Splitting and naming the textgrids 
split_and_name_textgrids()

NameError: name 'split_and_name_textgrids' is not defined

# Same process but for corrected_textgrid folder

In [ ]:
# file names 
cwd = os.getcwd()
corrected_textgrid_names_list = glob.glob(cwd + "/corrected_textgrid/*TextGrid")

In [ ]:
corrected_textgrid_names_list

In [ ]:
for file_name in corrected_textgrid_names_list:
    channel1, channel2 = get_file_names(corrected_textgrid_names_list[0])
    grid = textgrids.TextGrid(file_name)
    chan1, chan2 = combine_dfs(grid)
    # name_dfs correctly
    # save to csv
#     chan1.to_csv(cwd + '/split_wav_files_folder/' + channel1 + "TextGrid")
#     chan2.to_csv(cwd + '/split_wav_files_folder/' + channel2 + "TextGrid")

In [ ]:
chan1

In [123]:
cwd = os.getcwd()
textgrid_list = get_textgrids_for_each_speaker()
for file_name in textgrid_list:
    channel1, channel2 = get_file_names(file_name)
    grid = textgrids.TextGrid(file_name)
    chan1, chan2 = combine_dfs(grid)
    # name_dfs correctly
    # save to csv
    chan1.to_csv(cwd + '/split_wav_files_folder/' + channel1 + "TextGrid")
    chan2.to_csv(cwd + '/split_wav_files_folder/' + channel2 + "TextGrid")



NameError: name 'combine_dfs' is not defined

In [126]:
combine_dfs()

NameError: name 'combine_dfs' is not defined

In [135]:
combine_dfs()

NameError: name 'combine_dfs' is not defined